In [ ]:
from api import NFL, CLEAN
import os
import pyodbc
import sqlalchemy as sa
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 100)

In [ ]:
api_key = os.getenv("sportradar_apikey")
# api_key = os.getenv("sportradar_apikey2")
year = 2020
# year = 2019
# year = 2018
# year = 2017
# year = 2016
# year = 2015

In [ ]:
def get_season(api_key, year, access_level="trial", version="v7", language_code="en", format_="json", sleep_time=2, timeout=5):
    season = NFL(api_key, access_level=access_level, version=version, language_code=language_code, format_=format_, sleep_time=sleep_time, timeout=timeout)
    
    sql_server = "192.168.1.17\SQLEXPRESS"
    database = "FantasySidelines"
    sql_username = os.getenv("sql_username")
    sql_password = os.getenv("sql_password")

    connection_string = (
        "DRIVER={OBDC Driver 17 for SQL Server};"\
        f"SERVER={sql_server};"\
        f"Database={database};"\
        f"UID={sql_username};"\
        f"PWD={sql_password};"\
        "Trusted_Connection=no;"
    )

    connection_uri = f"mssql+pyodbc://?odbc_connection={urllib.parse.quote_plus(connection_string)}"
    engine = sa.create_engine(connection_uri, fast_executemany=True)
    conn = engine.connect()    
    
    schedule = pd.DataFrame()
    player_stats = pd.DataFrame()
    team_stats = pd.DataFrame()

    schedule_api = season.season_schedule(year)
    _schedule = CLEAN().schedule_clean(schedule_api)
    
    game_id = _schedule["game.id"].to_list()
    
    _player_stats = pd.DataFrame()
    _team_stats = pd.DataFrame()

    for game in game_id:
        statistics_api = season.game_statistics(game)
        
        _player = CLEAN().player_stats_clean(statistics_api)
        _player_stats = pd.concat([_player, _player_stats])
        
        _team = CLEAN().team_stats_clean(statistics_api)
        _team_stats = pd.concat([_team, _team_stats])

    schedule = pd.concat([_schedule, schedule])
    player_stats = pd.concat([_player_stats, player_stats])
    team_stats = pd.concat([_team_stats, team_stats])
    
    schedule_team = pd.concat([schedule, team_stats], axis=1)
    
    schedule_team.to_sql(f"teamStats{year}", engine, if_exists="append", index=False)
    plaer_stats.to_sql(f"playerStats{year}", engine, if_exists="append", index=False)
    
    return schedule_team, player_stats

In [ ]:
# import smtplib
# import time



# email = os.getenv("notification_email")
# email_password = os.getenv("notification_password")
# error = ""
# try:
#     pass
# except Exception as _ex:
#     error = repr(_ex)
# to = "cuddebtj@gmail.com"    
# if error != "":
#     subject = "Python Script Error"
#     body = f"Error raised: {error}\nTime to error: {(end_time-start_time)/60}"
# else:
#     subject = "Python Script Complete"
#     body = f"Script complete without errors.\nTime to error: {(end_time-start_time)/60}"
# email_text = f"From: {email}" \
#              f"To: {to}"\
#              f"Subject: {subject}"\
#              f"{body}"
# server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
# server. ehlo()
# server.login(email, email_password)
# server.sendmail(email, to, email_text)
# server.close()